In [1]:
import os
import sys
import shutil
import numpy as np
import pandas as pd
import pydicom as dicom
import matplotlib.pylab as plt
import SimpleITK as sitk
from datetime import datetime


In [2]:
%env SITK_SHOW_COMMAND "/home/andres/Downloads/Slicer-5.4.0-linux-amd64/Slicer"
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
dicom.config.convert_wrong_length_to_UN = True

env: SITK_SHOW_COMMAND="/home/andres/Downloads/Slicer-5.4.0-linux-amd64/Slicer"


In [3]:
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [4]:
from Utils import utils

In [5]:
config = utils.read_config()
config

{'common': {'paths': {'source_path_wd': 'E:/U-CAN-Lymfom_A',
   'source_path_bd': '/media/andres/T7 Shield/ucan_lymfom'}},
 'selection': {'filenames': {'rejected_folder_filename': 'Rejected_exams_from_U-CAN-Lymfom.xlsx',
   'sourcefiltered_folder_filename': 'SourceFiltered_exams_from_U-CAN-Lymfom.xlsx',
   'incomplete_folders_filename_initial': 'Excel_files/06_11_2023/No_PT_or_CT_exams_from_U-CAN-Lymfom_before_selection_process.xlsx',
   'incomplete_folders_filename_final': 'Excel_files/06_11_2023/No_PT_or_CT_exams_from_U-CAN-Lymfom_after_selection_process.xlsx',
   'selected_folders_beforefiltering_filename': 'Excel_files/06_11_2023/Selected_exams_after_filtering_from_U-CAN-Lymfom.xlsx',
   'selected_folders_afterfiltering_filename': 'Excel_files/06_11_2023/Selected_exams_before_filtering_from_U-CAN-Lymfom.xlsx',
   'final_selected_images_filename': 'Excel_files/06_11_2023/Final_Selected_exams_from_U-CAN-Lymfom.xlsx',
   'list_of_distorted_images_filename': 'Excel_files/06_11_2023/exa

In [6]:
#source_path_wd = "/media/andres/T7 Shield1/U-CAN-Lymfom_A/Selected_for_UCAN_project"
#source_path_bd = "/media/andres/T7 Shield/ucan_lymfom"

source_path_wd = config['common']['paths']['source_path_wd']
source_path_bd = config['common']['paths']['source_path_bd']

master_data_wd_filename = config['metadata']['filenames']['final_selected_images_filename']
master_data_bd_filename = config['metadata']['filenames']['final_selected_images_filename']


In [7]:
master_data_bd_filename

'Excel_files/06_11_2023/Finalized_dataset.xlsx'

In [8]:
final_selected_images_filename = config['selection']['filenames']['final_selected_images_filename']
final_selected_folders_bd = source_path_bd + '/' + final_selected_images_filename

In [10]:
# master_data for black disk
print("Reading master data for black disk")
master_data_bd = pd.read_excel(os.path.join(source_path_bd, master_data_bd_filename))
#master_data_wd[['system', 'npr', 'scan_date']]= master_data_wd['patient_directory'].str.split(pat='_|-', n=2, expand=True)
print(master_data_bd.shape)

master_data_bd.head(2)

Reading master data for black disk
(3646, 45)


,Unnamed: 0,Unnamed: 0_x,directory,source_directory,patient_directory,PET-CT_info,system,npr,scan_date,Unnamed: 0_y,dicom_img,patient_id,patient_age,patient_weight,patient_size,imgsz_x,imgsz_y,num_slices,voxsz_x,voxsz_y,slice_thickness,image_size,voxel_size,img_pos,img_orient,for_uid,att_corr,recons_method,image_type,aquisition_dt,aquisition_time,study_desc,series_desc,protocol,corr_img,modality,manufacturer,manufacturer_model,radiopharmaceutical,radiopharmaceutical_volume,radiopharmaceutical_start_time,radionuclide_total_dose,radionuclide_half_life,radionuclide_positron_fraction,radiopharmaceutical_start_date_time
0,0,3689,/media/andres/T7 Shield/ucan_lymfom/ASPTCTX0001_lpr385705046400-20140313/CT-20140313-152551-6.1_UAS-WB-FDG-3D-STANDARD_CT_RECON-3.750000mm,/media/andres/T7 Shield/ucan_lymfom,ASPTCTX0001_lpr385705046400-20140313,CT-20140313-152551-6.1_UAS-WB-FDG-3D-STANDARD_CT_RECON-3.750000mm,ASPTCTX0001,lpr385705046400,20140313,13743,1.2.840.113619.2.55.3.4240671514.332.1394175950.81.1,lpr385705046400,33.0,'69.0','1.73',512,512,215,0.976562,0.976562,3.75,"(512, 512, 215)","(0.976562, 0.976562, 3.75)","[-250.000, -250.000, 167.080]","[1.000000, 0.000000, 0.000000, 0.000000, 1.000000, 0.000000]",1.2.840.113619.2.55.3.4240671514.332.1394175949.830.13830.1,NaN,NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",'20140313','152608','FDG WB 1 h','STANDARD CT RECON','6.1 UAS-WB-FDG-3D',NaN,CT,'GE MEDICAL SYSTEMS','Discovery STE',NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,3690,/media/andres/T7 Shield/ucan_lymfom/ASPTCTX0001_lpr385705046400-20140313/PT-20140313-152728-UAS-WB-FDG-3D-UAS-WB-FDG-3D-AC-3.2700mm,/media/andres/T7 Shield/ucan_lymfom,ASPTCTX0001_lpr385705046400-20140313,PT-20140313-152728-UAS-WB-FDG-3D-UAS-WB-FDG-3D-AC-3.2700mm,ASPTCTX0001,lpr385705046400,20140313,13745,1.2.840.113619.2.131.4240671514.1394722651.463822,lpr385705046400,33.0,'69.0','1.73',128,128,247,3.906250,3.906250,3.27,"(128, 128, 247)","(3.90625, 3.90625, 3.27)","[-248.04687500000, -248.04687500000, 169.020004272460]","[1, 0, 0, 0, 1, -0]",1.2.840.113619.2.55.3.4240671514.332.1394175949.830.13830.1,"'measured,, 0.096000 cm-1,'",'3D IR',"['ORIGINAL', 'PRIMARY']",'20140313','154238','FDG WB 1 h','UAS-WB-FDG-3D-AC','UAS-WB-FDG-3D',"['DECY', 'ATTN', 'SCAT', 'DTIM', 'RAN', 'DCAL', 'SLSENS', 'NORM']",PT,'GE MEDICAL SYSTEMS','Discovery STE','FDG -- fluorodeoxyglucose','0.0','141400.00','259344912.0','6588.0','0.97000002861023','20140313141400.00'


In [ ]:
# master data from white disk
print("Readin master data from white disk")
master_data_wd = pd.read_excel(os.path.join(source_path_bd, master_data_wd_filename))
master_data_wd[['system', 'npr', 'scan_date']]= master_data_wd['patient_directory'].str.split(pat='_|-', n=2, expand=True)
print(master_data_wd.shape)

master_data_wd.head(2)

In [ ]:
misclassified_df = pd.read_excel(source_path_bd + '/MisclassifiedSelected_files.xlsx')
misclassified_df[['source_directory', 'patient_directory', 'PET-CT_info']] = misclassified_df['directory'].str.rsplit(pat='/', n=2, expand=True)
misclassified_patient_directory_list = misclassified_df.patient_directory.to_list()
misclassified_df.head()

In [ ]:
master_data_bd[np.logical_and(master_data_bd.num_slices >200,master_data_bd.num_slices < 250)].head(2) # 488

In [ ]:
exams_with_lt250 = master_data_wd[master_data_wd.num_slices < 250].patient_directory.to_list()
len(exams_with_lt250)

In [ ]:
master_data_wd_filtered = master_data_wd[~master_data_wd.patient_directory.isin(exams_with_lt250)].copy().reset_index()
master_data_wd_filtered.shape

In [ ]:
master_data_wd_filtered.patient_directory.nunique()

In [11]:
config_size = config['resampling']['image_size']
config_spacing = config['resampling']['voxel_spacing']
print('config_size: ', config_size)
print('config_spacing: ', config_spacing)

config_size:  [256, 256, 256]
config_spacing:  [2.0364201068878174, 2.0364201068878174, 3.0]


In [ ]:
sample_SUV_image = sitk.ReadImage('F:/SUV.nii.gz')
print(sample_SUV_image.GetSize())
config_spacing = list(sample_SUV_image.GetSpacing())
config_spacing

In [ ]:
#clip positive values at +3000
#set outside image 0 values to -1024

In [12]:
final_selected_folders_bd_sorted = final_selected_folders_bd.sort_values(by=["patient_directory"	,"PET-CT_info"], ascending=[True, False])
final_selected_folders_bd_sorted.head(5)

AttributeError: 'str' object has no attribute 'sort_values'

In [ ]:
exception_lst = []
resampled_SUV_CT = {'patient_directory' : [],
                    'SUV': [],
                    'CT' : [],
                    'new_size' : []}
                    #'SUV_arr': []}

for index, row in final_selected_folders_bd_sorted.head(70).iterrows():
    #print(index, row['patient_directory'], row['PET-CT_info'])
    #if index == 2:
    #    break
    
    #create patient directories
    npr_directories = '/media/andres/T7/Sambit/Other_Projects/UCAN/Resampled_SUV_CT/' + str(row['npr']) + '_SUV_CT/'
    #npr_directories = 'F:/ucan_lymfom/Resampled_SUV_CT/Sample24Oct2023/' + str(row['npr']) + '_SUV_CT/'
    if not os.path.exists(npr_directories):
        os.makedirs(npr_directories)
    
    #create scan date directoriesmaster_data_wd_filtered.head(2)
    scan_date_directories = '/media/andres/T7/Sambit/Other_Projects/UCAN/Resampled_SUV_CT/' + str(row['npr']) + '_SUV_CT/' + str(row['scan_date']) + '/'
    #scan_date_directories = 'F:/ucan_lymfom/Resampled_SUV_CT/Sample24Oct2023/' + str(row['npr']) + '_SUV_CT/' + str(row['scan_date']) + '/'
    if not os.path.exists(scan_date_directories):
        os.makedirs(scan_date_directories)
        
    if row['modality']=='PT':

        resampled_SUV_CT['patient_directory'].append(row['patient_directory'])

        vol_img = utils.read_dicom(row['directory'])
        image_size = vol_img.GetSize()

        new_size = [config_size[0], config_size[1], image_size[2]]
        resampled_SUV_CT['new_size'].append(new_size)
        #print(new_size)   

        suv, estimated, raw,spacing,origin,direction = utils.compute_suv(vol_img, PatientWeight, AcquisitionTime, RadiopharmaceuticalStartTime, RadionuclideHalfLife, RadionuclideTotalDose)
        suv_img = sitk.GetImageFromArray(suv)

        suv_img = sitk.Resample(vol_img, new_size, sitk.Transform(), sitk.sitkNearestNeighbor, #sitk.sitkLinear, 
                            vol_img.GetOrigin(), config_spacing, vol_img.GetDirection(), 0,
                            vol_img.GetPixelID())  
        
        PatientWeight = float(row['patient_weight'].replace("'", "").replace(" ", ""))

        AcquisitionTime = str(row['aquisition_time'])
        AcquisitionTime = AcquisitionTime.replace("'","").strip()

        # Start Time for the Radiopharmaceutical Injection
        RadiopharmaceuticalStartTime = row['radiopharmaceutical_start_time'].split('.')[0].replace("'","").strip()

        # Half Life for Radionuclide # seconds
        RadionuclideHalfLife = row['radionuclide_half_life'].replace("'","").strip()

        # Total dose injected for Radionuclide
        RadionuclideTotalDose = row['radionuclide_total_dose'].replace("'","").strip()

        # suv, estimated, raw,spacing,origin,direction = utils.compute_suv(vol_img, PatientWeight, AcquisitionTime, RadiopharmaceuticalStartTime, RadionuclideHalfLife, RadionuclideTotalDose)
        # suv_img = sitk.GetImageFromArray(suv)

        # suv_img.SetSpacing(config_spacing)

        #suv_img = sitk.Resample(suv_img, new_size, sitk.Transform(), sitk.sitkLinear,
        #                    suv_img.GetOrigin(), config_spacing, suv_img.GetDirection(), 0,
        #                   suv_img.GetPixelID())
        
        resampled_SUV_CT['SUV'].append(row['PET-CT_info'])
        resampled_SUV_CT['CT'].append('')
        #resampled_SUV_CT['SUV_arr'].append(sitk.GetArrayFromImage(suv_img))

        save_path= '/media/andres/T7/Sambit/Other_Projects/UCAN/Resampled_SUV_CT/' + str(row['npr']) + '_SUV_CT/' + str(row['scan_date']) + '/' + 'SUV'
        #save_path= 'F:/ucan_lymfom/Resampled_SUV_CT/Sample24Oct2023/' + str(row['npr']) + '_SUV_CT/' + str(row['scan_date']) + '/' + 'SUV'

        utils.save_as_gz(suv_img, save_path+'.nii.gz')
        
resampled_SUV_CT_df = pd.DataFrame(resampled_SUV_CT)
#resampled_SUV_CT_df.head(2)

In [ ]:
npr126347730283

In [ ]:
master_data_wd_filtered_sorted[master_data_wd_filtered_sorted.npr=="npr126347730283"]

In [ ]:
img1 = utils.read_dicom(r"C:\Users\NickosKal\Desktop\University\Semester 3\Project in Image Analysis\UCAN-PET-CT-image-data-handling-pipeline\test\PT-20170807-105022-6.40_WB_FDG_CT_VENFAS-QCFX-S_350-2.7900mm")
image_size = img1.GetSize()

new_size = [config_size[0], config_size[1], image_size[2]]

img1.GetSpacing(), img1.GetSize(), new_size

In [ ]:
#img1 = sitk.Resample(img1, new_size, sitk.Transform(), sitk.sitkNearestNeighbor, #sitk.sitkLinear, 
#                            img1.GetOrigin(), config_spacing, img1.GetDirection(), 0,
#                            img1.GetPixelID()) 

suv, estimated, raw,spacing,origin,direction = utils.compute_suv(img1, 92.0, '110032', '095000.00', '6586.2001953125', '361595072.0')
suv_img= sitk.GetImageFromArray(suv)
#suv_img.SetSpacing(config_spacing)
suv_img.GetSpacing(), suv_img.GetSize(), suv_img.GetOrigin()


In [ ]:
suv_img = sitk.Resample(suv_img, new_size, sitk.Euler3DTransform(), sitk.sitkNearestNeighbor, #sitk.sitkLinear, 
                            suv_img.GetOrigin(), config_spacing, suv_img.GetDirection(), 0,
                            suv_img.GetPixelID()) 
suv_img.GetSpacing(), suv_img.GetSize(), suv_img.GetOrigin()
utils.save_as_gz(suv_img, r'UCAN-PET-CT-image-data-handling-pipeline'+'SUV.nii.gz')


In [ ]:
suv_arr=sitk.GetArrayFromImage(suv_img)
suv_arr.max(), suv_arr.min(), np.unique(suv_arr)

In [ ]:
ct_img= utils.read_dicom(r"C:\Users\NickosKal\Desktop\University\Semester 3\Project in Image Analysis\UCAN-PET-CT-image-data-handling-pipeline\test\CT-20170807-110734-6.40_WB_FDG_CT_VENFAS-WB_Venfas_3_ax-3.000000mm")
ct_img.GetSpacing(), ct_img.GetSize()

In [ ]:
print(new_size)
ct_img = sitk.Resample(ct_img, new_size, sitk.Euler3DTransform(), sitk.sitkNearestNeighbor, #sitk.sitkLinear, 
                            ct_img.GetOrigin(), config_spacing, ct_img.GetDirection(), -1024,
                            ct_img.GetPixelID()) 
ct_img.GetSpacing(), ct_img.GetSize(), ct_img.GetOrigin()#,np.unique(ct_img)

In [ ]:
suv_img1 = sitk.Resample(suv_img, new_size, sitk.Transform(), sitk.sitkNearestNeighbor, #sitk.sitkLinear, 
                            suv_img.GetOrigin(), config_spacing, suv_img.GetDirection(), 0,
                            suv_img.GetPixelID()) 
#suv_img1.SetSpacing(ct_img.GetOrigin())
suv_img1.SetDirection(ct_img.GetDirection())
suv_img1.SetOrigin(ct_img.GetOrigin())
suv_img1.GetSpacing(), suv_img1.GetSize(), suv_img1.GetOrigin()

In [ ]:
np.unique(sitk.GetArrayFromImage(suv_img1))

In [ ]:
np.unique(sitk.GetArrayFromImage(ct_img))

In [ ]:
ct_img.GetOrigin()

In [ ]:
ct_img_arr = sitk.GetArrayFromImage(ct_img)
ct_img_arr[ct_img_arr < -1024] = -1024
ct_img = sitk.GetImageFromArray(ct_img_arr)

In [ ]:
utils.save_as_gz(suv_img, r'UCAN-PET-CT-image-data-handling-pipeline_'+'SUV.nii.gz')
utils.save_as_gz(ct_img, r'UCAN-PET-CT-image-data-handling-pipeline_'+'CT.nii.gz')

In [ ]:
img = sitk.ReadImage("/media/andres/T7 Shield/U-CAN-Lymfom_A/Sambit/Resampled_SUV_CT/npr100589333593_SUV_CT/20190205/SUV.nii.gz")
img.GetSpacing(), img.GetSize()

In [ ]:
img = sitk.ReadImage("/media/andres/T7 Shield/U-CAN-Lymfom_A/Sambit/Resampled_SUV_CT/npr100589333593_SUV_CT/20190205/CT.nii.gz")
img.GetSpacing(), img.GetSize()

In [ ]:
for index, row in master_data_wd_filtered_sorted.head(70).iterrows():
    #print(index, row['patient_directory'], row['PET-CT_info'])
    #if index == 2:
    #    break
    
    if row['modality']=='CT':  
        #print(row[['patient_directory']])      
        resampled_SUV_CT_df.loc[resampled_SUV_CT_df['patient_directory'] == row['patient_directory'], 'CT'] = row['PET-CT_info']

        vol_img = utils.read_dicom(row['directory'])
        image_size = vol_img.GetSize()

        #new_size = resampled_SUV_CT_df.loc[resampled_SUV_CT_df['patient_directory'] == row['patient_directory'], 'new_size'].tolist()[0]

        suv_img = sitk.ReadImage('/media/andres/T7/Sambit/Other_Projects/UCAN/Resampled_SUV_CT/' + str(row['npr']) + '_SUV_CT/' + str(row['scan_date']) + '/' + 'SUV' + '.nii.gz')
        new_size = suv_img.GetSize()
        
        #suv_img = sitk.ReadImage('F:/ucan_lymfom/Resampled_SUV_CT/Sample24Oct2023/' + str(row['npr']) + '_SUV_CT/' + str(row['scan_date']) + '/' + 'SUV' + '.nii.gz')
        
        vol_img = sitk.Resample(vol_img, new_size, sitk.Transform(), sitk.sitkNearestNeighbor, #sitk.sitkLinear,
                            vol_img.GetOrigin(), config_spacing, vol_img.GetDirection(), -1024,
                            vol_img.GetPixelID())
        
        vol_img.SetOrigin(suv_img.GetOrigin())
        
        vol_img_arr = sitk.GetArrayFromImage(vol_img)
        vol_img_arr[vol_img_arr < -1024] = -1024

        vol_img = sitk.GetImageFromArray(vol_img_arr)

        #vol_img = sitk.Clamp(vol_img,upperBound=3000)

        save_path= '/media/andres/T7/Sambit/Other_Projects/UCAN/Resampled_SUV_CT/' + str(row['npr']) + '_SUV_CT/' + str(row['scan_date']) + '/' + 'CT'
        #save_path= 'F:/ucan_lymfom/Resampled_SUV_CT/Sample24Oct2023/' + str(row['npr']) + '_SUV_CT/' + str(row['scan_date']) + '/' + 'CT'

        utils.save_as_gz(vol_img, save_path+'.nii.gz')
        
#resampled_SUV_CT_df.head(2)

In [ ]:
### Correct origin

In [ ]:
sample_CT = sitk.ReadImage('E:/Sambit/Other_Projects/UCAN/Resampled_SUV_CT/lpr415675513429_SUV_CT/20190201/CT.nii.gz')
sample_SUV = sitk.ReadImage('E:/Sambit/Other_Projects/UCAN/Resampled_SUV_CT/lpr415675513429_SUV_CT/20190201/SUV.nii.gz')

In [ ]:
print(sample_CT.GetSize())
print(sample_CT.GetSpacing())
print(sample_CT.GetOrigin())
print(sample_CT.GetDirection())
print(sample_SUV.GetSize())
print(sample_SUV.GetSpacing())
print(sample_SUV.GetOrigin())
print(sample_SUV.GetDirection())

In [ ]:
#sample_CT = sitk.Clamp(sample_CT,upperBound=3000)
#utils.save_as_gz(sample_CT, save_path+'2.nii.gz')
sample_CT_arr = sitk.GetArrayFromImage(sample_CT)
print(sample_CT_arr.max())

(sample_CT_arr.min())

In [ ]:
sample_CT_arr.size

In [ ]:
sample_CT_arr[sample_CT_arr==0].size

In [ ]:

sample_CT = sitk.ReadImage('/media/andres/T7/Sambit/Other_Projects/UCAN/Resampled_SUV_CT/npr100169878450_SUV_CT/20130412/CT.nii.gz')
sample_CT_arr = sitk.GetArrayFromImage(sample_CT)
print(sample_CT_arr.max())

(sample_CT_arr.min())

sample_CT_arr[sample_CT_arr < -1024] = -1024

sample_CT_arr.min()

In [ ]:
plt.imshow(sample_CT_arr.transpose(1,2,0))